In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
import os
pasta_campi = 'campi/'
lista_campi = []
lista_priori = []
for diretorios, subpastas, arquivos in os.walk(pasta_campi):
    lista_campi = [arquivo for arquivo in arquivos]

pasta_priori = 'priori/'
for diretorios, subpastas, arquivos in os.walk(pasta_priori):
    lista_priori = [arquivo for arquivo in arquivos]
    
# print(lista_campi)
# print(f'\n\n{lista_priori}')
dicionario = {'CAFI.xlsx': 'AFI.xlsx'}
"""dicionario = {
    'CAFI.xlsx': 'AFI.xlsx',
    'CBAR.xlsx': 'BAR.xlsx',
    'CBJA.xlsx': 'BJA.xlsx',
    'CGAR.xlsx': 'GAR.xlsx',
    'CPES.xlsx': 'PES.xlsx',
    'CPLT.xlsx': 'PLT.xlsx',
    'CABL.xlsx': 'ABR.xlsx',
    'CCAR.xlsx': 'CAR.xlsx',
    'CCSA.xlsx': 'CAB.xlsx',
    'CIGR.xlsx': 'IGA.xlsx',
    'CIPJ.xlsx': 'IPJ.xlsx',
    'CJBG.xlsx': 'JAB.xlsx',
    'COLI.xlsx': 'OLI.xlsx',
    'CPLT.xlsx': 'PLT.xlsx',
    'CPMR.xlsx': 'PRM.xlsx',
    'CREC.xlsx': 'REC.xlsx',
    'CVSA.xlsx': 'VSA.xlsx',
    'REIFPE.xlsx': 'REI.xlsx',
    
}"""
#ARQUIVO DESLIZANTE caruaru softwares em ccsa  modulo voitaico e softwares em gar
dicionarioum = {
    'CREC.xlsx': 'REC.xlsx',
}

lista_drop = ['MÓDULO DIDÁTICO,', 'PLACA DE GESSO ACARTONADO', 'PORTA PARA DIVISÓRIA', 'Conjunto de guarda corpo medindo', 
             'PAR de corrimão em ferro', 'COBERTURA LEVE', 'KIT SALA MODULAR', 'CADEIRA ESCOLAR EM RESINA', 
             'CARTEIRA ESCOLAR - EM RESINA', 'CADEIRA PLASTICA UNIVERSAL',
             'CADEIRA ESCOLAR', 'CADEIRA COMUM. CADEIRA DE PLÁSTICO', 'MESA QUADRADA DE PLÁSTICO', 
             'POLTRONA PRANCHETA EM RESINA', 'Cadeira comum de plástico', 'Cadeira Escolar fixa', 'CARTEIRA UNIVERSITÁRIA',
             'CADEIRA EM POPIPROPILENO', 'CBAR - CARTEIRA', 'CBAR - CADEIRA', 'CADEIRA UNIVERSITARIA', 'CADEIRA ACADÊMICA',
             'CADEIRA COM PRANCHETA', 'Forro Isolante Térmico', 'Placa de Gesso Acartonado', 'Espuma, aplicação revestimento',
             'Carteiras escolares', 'CADEIRA COMUM DE PLÁSTICO', 'CADEIRA PLASTICA BRANCA', 'Poltrona Escolar com prancheta',
             'CARTEIRA ESCOLAR', 'CADEIRA PLÁSTICA SEM BRAÇO NA COR VERDE ESCURO', 'Carteira Escolar',
             'CADEIRA ESCOLAR/UNIVERSITARIA', 'CIPO - CADEIRA', 'Cadeira comum , empilhável', 'DIVISÓRIA EM EUCATEX',
             'CADEIRA PLÁSTICA DE POLIPROPILENO', 'Carteira escolar com prancheta frontal', 'Mesa quadrada de plástico',
             'CADEIRA FIXA, EMPILHÁVEL', 'CADEIRA PLÁSTICA EMPILHÁVEL', 'CPES - CADEIRA FIXA','Cadeira comun de plástico, sem braço'
             'CPES - CADEIRA EMPILHAVEL', 'CPES - CARTEIRA', 'CADEIRA FIXA EMPILHÁVEL EM POLIPROPILENO', 'Poltrona universitária',
             'CADEIRA FIXA , EMPILHÁVEL EM POLIPROPILENO', 'MESA ESCOLAR, COR VERDE', 'MESA ESCOLAR ALUNO', 'CARTEIRA BRACO-MESA',
             'TIPO UNIVERSITARIA REF 501', 'CARTEIRA UNIVERSITARIA', 'C/PRANCHETA VINIL ESTRUT.ACO TUBULAR', 'PAINEL PISO TETO CEGO',
             'TUBO CONECTOR PAINEL PISO TETO', 'COMPONENTE FIXO DE ARQUIVAMENTO DESLIZANTE', 'COMPONENTE RETRÁTIL DE ARQUIVAMENTO',
             'PAINEL PISO TETO', 'Cadeira de plástico, sem braço', 'Mesa de plástico, quadrada, empilhável', 'Cadeira escolar com prancheta frontal',
             'Persiana vertical para janela', 'Cadeira Escolar fixa - Cor verde', 'Cadeira de plástico, empilhável', 'Carteira Escolar de Braço',
              'CADEIRA ESCOLAR EM RESINA, MARCA SK']


# print(dicionario.keys())
# print(dicionario.values())

In [10]:
import pandas as pd
from datetime import datetime, timezone

def montar_consolidacao(dic):

    for key, value in  dic.items():
        
        campus = pd.read_excel(f'campi/{key}')

        #converte pra date utilizando o timezone utc=False não roda
        campus['data_entrada'] = pd.to_datetime(campus['data_entrada'], utc=True)
        # remove a timezone deixando so a data mas vira type obect novamente
        campus['data_entrada'] = campus['data_entrada'].dt.date
        # converte em date novamente agora sem o timezone
        campus['data_entrada'] = pd.to_datetime(campus['data_entrada'])

        campus = campus[campus['data_entrada'] < '2021-05-01']
        # campus['valor'] = pd.to_numeric(campus['valor'])

        # display(campus.info())
        campus = campus[['numero', 'categoria_codigo', 'descricao', 'data_entrada']]
#         display(campus)

        priori = pd.read_excel(f'priori/{value}')
        # display(priori)
        priori = priori[['N° Ativo', 'Nome do custo', 'Descrição']]
        priori = priori.rename(columns={'N° Ativo': 'numero'})
#         display(priori)
        
        compara = pd.merge(campus, priori, on='numero', how='outer', indicator=True)
        compara.to_excel(f'consolidacao/cosolidacao_{key}', index=False)

In [11]:
def dropar_descricao(dic, descircao_suap, descricao_priori, lista):
    
    for key, value in dic.items():
        
        campus = pd.read_excel(f'campi/{key}')
        priori = pd.read_excel(f'priori/{value}')
        
        for item in lista:
    #         data_drop = data_frame[data_frame[coluna].str.contains(item)] 
            campus = campus[campus[descircao_suap].str.contains(item)==False] #todas as linhas que não possuem o item
            priori = priori[priori[descricao_priori].str.contains(item)==False]
    #         print(f'{item} - {len(data_drop)}')
#         print((len(priori)/len(campus))*100)
            return priori, campus
#         print(len(priori))

# print(suap_total.info())

In [12]:
def contar_planilhas_sem_cadeiras(dic):
    
#     dic = dropar_descricao(dic, descircao_suap, descricao_priori, lista)
    
    df_quant = pd.DataFrame()
    nome_campi = []
    lista_suap = []
    lista_priori = []

    for key, value in  dic.items():
        campus = pd.read_excel(f'campi/{key}')
        priori = pd.read_excel(f'priori/{value}')
        
        for item in lista_drop:
    #         data_drop = data_frame[data_frame[coluna].str.contains(item)] 
            campus = campus[campus['descricao'].str.contains(item)==False] #todas as linhas que não possuem o item
            priori = priori[priori['Descrição'].str.contains(item)==False]
            
        conta_campus = len(campus)
        lista_suap.append(conta_campus)
        nome_campi.append(key)

        conta_priori = len(priori)
        lista_priori.append(conta_priori)

    df_quant['Nome'] = nome_campi
    df_quant['Nome'] = df_quant['Nome'].str[0:4]
    df_quant['Suap'] = lista_suap
    df_quant['Priori'] = lista_priori
    df_quant['Diferenca'] = df_quant['Suap'] - df_quant['Priori']
    df_quant['porcentagem'] = (df_quant['Priori'] / df_quant['Suap']) * 100
    display(df_quant)
    df_quant.to_excel('quantidades_sem_cadeiras.xlsx', index=False)

In [13]:
def contar_planilhas(dic):
    
    df_quant = pd.DataFrame()
    nome_campi = []
    lista_suap = []
    lista_priori = []

    for key, value in  dic.items():
        
        
        campus = pd.read_excel(f'campi/{key}')
        
        conta_campus = len(campus)
        lista_suap.append(conta_campus)
        nome_campi.append(key)
        
        priori = pd.read_excel(f'priori/{value}')
        conta_priori = len(priori)
        lista_priori.append(conta_priori)

    df_quant['Nome'] = nome_campi
    df_quant['Nome'] = df_quant['Nome'].str[0:4]
    df_quant['Suap'] = lista_suap
    df_quant['Priori'] = lista_priori
    df_quant['Diferenca'] = df_quant['Suap'] - df_quant['Priori']
    df_quant['porcentagem'] = (df_quant['Priori'] / df_quant['Suap']) * 100
    display(df_quant)
    df_quant.to_excel('quantidades.xlsx', index=False)

In [14]:
# montar_consolidacao(dicionario)
contar_planilhas(dicionario)
contar_planilhas_sem_cadeiras(dicionario)

,Nome,Suap,Priori,Diferenca,porcentagem
0,CAFI,9884,8354,1530,84.520437


,Nome,Suap,Priori,Diferenca,porcentagem
0,CAFI,8593,8220,373,95.659258
